# Лабораторна робота 8

__Знайдіть к-ть різних розфарбувань кубика Рубика 2х2х2, використовуючи 10 кольорів.__


![rubik cube](../docs/labs_assets/rubik_cube.jpg)

Два розфарбування називатимуться однаковими, якщо існує послідовність рухів кубика (тобто за допомогою симетрій кубика) рубика, яка одне розфарбування переводить в інше. 

Не обовʼязково використовувати всі 10 кольорів, тобто допустимо пофарбувати весь кубик в один колір. 

___Приклад___: існує 183 різних розфарбувань, використовуючи 2 кольори. 

In [2]:
G = SymmetricGroup(24)
g1 = G('(1, 2, 4, 3)(5, 24, 9, 7)(6, 23, 10, 8)');
...

H = G.subgroup([g1, g2, g3, g4, g5, g6])
H.order()
# 88179840

88179840

Для вирішення цієї задачі можна використовувати Burnside's Lemma, яка дозволяє обчислити кількість орбіт при дії групи симетрій на множину об'єктів. У цьому випадку група симетрій кубика Рубика 2x2x2 має 24 елементи (це всі можливі рухи кубика).

Кроки для розв'язку:

- Кубик Рубика 2x2x2 має 8 вершин (кутів), кожну з яких можна пофарбувати одним із 10 кольорів.
- Для кожного симетричного перетворення кубика визначаємо, скільки кольорів залишається незмінним після цього перетворення.
- Застосовуємо Burnside's Lemma для підрахунку кількості різних орбіт (тобто розфарбувань).

In [13]:
# from sage.all import *

In [18]:
from sage.all import *

G = SymmetricGroup(24)

# симетрії кубика через пермутації (почали рахувати ще на парі)
g1 = G('(1, 2, 4, 3)(5, 24, 9, 7)(6, 23, 10, 8)')
g2 = G('(7, 8, 14, 13)(3, 9, 18, 12)(4, 15, 17, 6)')
g3 = G('(9, 10, 16, 15)(4, 24, 20, 14)(2, 22, 18, 8)')
g4 = G('(11, 5, 6, 12)(1, 7, 17, 21)(3, 13, 19, 23)')
g5 = G('(23, 24, 22, 21)(1, 10, 20, 11)(2, 16, 19, 5)')
g6 = G('(20, 19, 17, 18)(22, 11, 13, 15)(21, 12, 14, 16)')

H = G.subgroup([g1, g2, g3, g4, g5, g6])

# визначення циклових типів для кожної симетрії, тобто к-сті
cycletypes = {}  

for C in H.conjugacy_classes():
    g = C.representative()
    cycle_structure = g.cycle_type()
    cycletypes[tuple(cycle_structure)] = cycletypes.get(tuple(cycle_structure), 0) + C.cardinality()

# обчислення к-сті різних кольорів
def Burnside(cycle_count, colours, H):
    modG = H.order()
    sum = 0
    for key, value in cycle_count.items():
        power = len(key)
        sum += colours**power * value
    return sum / modG


order = H.order()
colourings_2 = Burnside(cycletypes, 2, H)
colourings_10 = Burnside(cycletypes, 10, H)

print("Rubik's Cube Symmetry Information:")
print("-" * 40)
print(f"order of the symmetry group: {order}")
print(f"distinct colourings with 2 colours: {colourings_2}")
print(f"distinct colourings with 10 colours: {colourings_10}")

Rubik's Cube Symmetry Information:
----------------------------------------
order of the symmetry group: 88179840
distinct colourings with 2 colours: 183
distinct colourings with 10 colours: 12395526079546335


Симетрії кубика: задаємо шість перестановок елементів кубика через пермутації в групі симетрій $G$:

- __𝑔1=(1,2,4,3)(5,24,9,7)(6,23,10,8)__
- __𝑔2=(7,8,14,13)(3,9,18,12)(4,15,17,6)__
- __𝑔3=(9,10,16,15)(4,24,20,14)(2,22,18,8)__ 
- __𝑔4=(11,5,6,12)(1,7,17,21)(3,13,19,23)__ 
- __𝑔5=(23,24,22,21)(1,10,20,11)(2,16,19,5)__ 
- __𝑔6=(20,19,17,18)(22,11,13,15)(21,12,14,16)__

Вони створюють підгрупу $H$ групи $G$.

Ми збираємо всі циклові типи і їх к-сть у підгрупі $H$ в змінну __cycletypes__ за допомогою наступного коду:

In [15]:
cycletypes = {}
for C in H.conjugacy_classes():
    g = C.representative()
    cycle_structure = g.cycle_type()
    cycletypes[tuple(cycle_structure)] = cycletypes.get(tuple(cycle_structure), 0) + C.cardinality()

__Burnside's Lemma__  дозволяє знайти кількість орбіт групи (у нашому випадку — к-сть різних розфарбувань кубика) за допомогою середнього арифметичного к-сті фіксованих елементів для кожної перестановки в групі.

Під дією кожної перестановки $g$ визначаємо к-сть фіксованих розфарбувань кубика, що не змінюються цією перестановкою. К-сть таких розфарбувань залежить від к-сті циклів у перестановці.

Якщо перестановка складається з $k$ незалежних циклів, то для кожного циклу ми можемо вибрати один колір з $m$ можливих. Таким чином, для кожної перестановки к-сть фіксованих розфарбувань буде дорівнювати $m^k$, де $k$ — це кількість циклів у перестановці.

Реалізація __Burnside's Lemma__ виглядає так:

In [16]:
def Burnside(cycle_count, colours, H):
    modG = H.order()
    sum = 0
    for key, value in cycle_count.items():
        power = len(key)
        sum += colours**power * value
    return sum / modG